In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms
import time
import pickle
import numpy as np
import matplotlib.pyplot as plt
import torch.optim.lr_scheduler as lr_scheduler

In [25]:
class LumpyDataset(Dataset):
    def __init__(self, images, labels) -> None:
        self.X = images
        self.y = labels
        # function for images transformations
        # TODO: try AutoAugment(AutoAugmentPolicy.IMAGENET)
        self.random_transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.RandomRotation(30),
            transforms.RandomHorizontalFlip(),
            transforms.ColorJitter(
                brightness=0.1, contrast=0.1),
            transforms.RandomAffine(
                degrees=(0, 30),
                translate=(0, 0.2),
                scale=(0.9, 1)
            ),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.X)
    
    def get_labels(self):
        return self.y

    def __getitem__(self, index):
        num_augment = 8
        augmented_batch = []
        # creating augmented data
        for i in range(num_augment):
            new_item = self.random_transform(self.X[index])
            augmented_batch.append(new_item)
        # labels with one-hot encoding
        label = torch.Tensor([self.y[index]])

        new_labels = [label, label, label, label, label, label, label, label]

        return torch.stack(augmented_batch), torch.stack(new_labels)

In [26]:
# loading all dataloaders
with open("./variables/train_loader.pickle", "rb") as f:
    train_loader = pickle.load(f)
with open("./variables/valid_loader.pickle", "rb") as f:
    valid_loader = pickle.load(f)
with open("./variables/test_loader.pickle", "rb") as f:
    test_loader = pickle.load(f)

In [27]:
# loading pretrained ResNet50 model
weights = torchvision.models.ResNet50_Weights.DEFAULT
resnet = torchvision.models.resnet50(weights=weights)

In [28]:
# unfreezing model's parameters
for param in resnet.parameters():
    param.requires_grad = True

n_inputs = resnet.fc.in_features
# modifying last layer of model
resnet.fc = nn.Sequential(nn.Linear(n_inputs, 1024),
                          nn.SELU(),
                          nn.Dropout(p=.4),
                          nn.Linear(1024, 1024),
                          nn.SELU(),
                          nn.Dropout(p=0.4),
                          nn.Linear(1024, 1))

for name, child in resnet.named_children():
    for name2, params in child.named_parameters():
        params.requires_grad = True

In [29]:
# use GPU if available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [30]:

resnet.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [31]:
# defining loss function and optimizer
lossfunc = nn.BCEWithLogitsLoss().to(device)
optimizer = torch.optim.Adam(resnet.parameters(), lr=3e-4)

train_losses, test_losses, train_correct, test_correct = [], [], [], []

scheduler = lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.6, total_iters=15)

In [32]:
def save_checkpoint(state, epoch, acc, foldername="./best_models/"):
    torch.save(state, foldername + f"epoch_{epoch}_acc_{acc}")

In [33]:
class EarlyStopping:
    def __init__(self, max_counter, threshold) -> None:
        self.stop_counter = 0
        self.last_best = 0
        self.max_counter = max_counter
        self.threshold = threshold
    
    def stop_learning(self, accuracy):
        if accuracy <= self.last_best or abs(accuracy - self.last_best) <= self.threshold:
            self.stop_counter += 1
            print("MODEL DID NOT IMPROVE FROM", self.last_best)
        else:
            self.stop_counter = 0

        if self.stop_counter == self.max_counter:
            return True
        return False

In [34]:
batch_size = 8
sample_size = 8
def train_the_model(model, epochs):
    start_time = time.time()
    for epoch_i in range(epochs):
        train_corr, test_corr = 0, 0
        total_num = 0

        epoch_start = time.time()
        batch_corr = 0
        for batch_num, (X, y) in enumerate(train_loader):
            total_num += sample_size * batch_size
            X, y = X.to(device), y.to(device)

            y_pred = resnet(X.view(-1, 3, 225, 225))
            loss = lossfunc(y_pred, y.view(batch_size * sample_size, -1)) # 16 samples in batch, 3 images in one sample
            # find predicted labels
            predicted = y_pred.view(batch_size * sample_size) > .5

            # calculate the amount of correct predictions in batch
            batch_corr += (predicted.long() == y.view(batch_size*sample_size)).sum()
            # backprop
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        train_corr = batch_corr.item() / (total_num)
        epoch_end = time.time() - epoch_start
        print(f"Epoch {epoch_i + 1} | Batch {(batch_num + 1) * batch_size}\nAccuracy: {train_corr:2.2f} | Loss: {loss.item():2.4f} | Duration: {epoch_end / 60:.2f} minutes")

        train_b = batch_num
        train_losses.append(loss)
        train_correct.append(train_corr)

        X, y = None, None

        with torch.no_grad():
            total_num = 0
            for batch_num, (X, y) in enumerate(valid_loader):
                total_num += sample_size * batch_size
                X, y = X.to(device), y.to(device)

                y_val = resnet(X.view(-1, 3, 225, 225))
                predicted = y_val.view(batch_size * sample_size) > .5

                test_corr += (predicted.long() == y.view(batch_size * sample_size)).sum()
        
        loss = lossfunc(y_val, y.view(batch_size * sample_size, 1))

        val_acc = test_corr.item() / (total_num)

        print(f"Validation Accuracy: {val_acc * 100:2.2f} | Validation loss: {loss.item():2.4f}\n")

        save_checkpoint(resnet.state_dict(),
                        epoch_i + 1,
                        val_acc)

        test_b = batch_num
        test_losses.append(loss)
        test_correct.append(val_acc)

        before_lr = optimizer.param_groups[0]["lr"]
        scheduler.step()
        after_lr = optimizer.param_groups[0]["lr"]

        print(f"LR {before_lr * 10000: .3f}-e4 -> {after_lr * 10000: .3f}-e4")

        if earlystopping.stop_learning(val_acc):
            break

    end_time = time.time() - start_time

    print(f"\nTraining Duration: {end_time / 60:.2f} minutes")
    print(f"GPU memory used: {torch.cuda.memory_allocated()} kb")
    print(f"GPU memory cached: {torch.cuda.memory_cached()} kb")

    return model, train_losses, test_losses, train_correct, test_correct

In [35]:
epochs = 5

earlystopping = EarlyStopping(3, 0.2)

best_prec1 = 2

batch_num, train_b, test_b = None, None, None

model, train_losses, test_losses, train_correct, test_correct = train_the_model(resnet, epochs)

Epoch 1 | Batch 520
Accuracy: 0.85 | Loss: 0.3415 | Duration: 10.60 minutes
Validation Accuracy: 77.15 | Validation loss: 0.4835

LR  3.000-e4 ->  2.920-e4
Epoch 2 | Batch 520
Accuracy: 0.89 | Loss: 0.2303 | Duration: 12.15 minutes
Validation Accuracy: 91.80 | Validation loss: 0.0686

LR  2.920-e4 ->  2.840-e4
Epoch 3 | Batch 520
Accuracy: 0.91 | Loss: 0.4360 | Duration: 12.06 minutes
Validation Accuracy: 91.99 | Validation loss: 0.1975

LR  2.840-e4 ->  2.760-e4
Epoch 4 | Batch 520
Accuracy: 0.96 | Loss: 0.1942 | Duration: 12.03 minutes
Validation Accuracy: 86.72 | Validation loss: 0.6745

LR  2.760-e4 ->  2.680-e4
Epoch 5 | Batch 520
Accuracy: 0.94 | Loss: 0.0315 | Duration: 10.92 minutes
Validation Accuracy: 88.67 | Validation loss: 0.3657

LR  2.680-e4 ->  2.600-e4

Training Duration: 59.16 minutes
GPU memory used: 1225899008 kb
GPU memory cached: 7650410496 kb


c:\Users\husai\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\cuda\memory.py:416: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  warnings.warn(


In [38]:
best_model_dict = torch.load("./best_models/epoch_5_acc_0.88671875")
best_model = torchvision.models.resnet50(weights=weights)
best_model.fc = nn.Sequential(nn.Linear(n_inputs, 1024),
                          nn.SELU(),
                          nn.Dropout(p=.4),
                          nn.Linear(1024, 1024),
                          nn.SELU(),
                          nn.Dropout(p=0.4),
                          nn.Linear(1024, 1))
best_model.to(device)
best_model.load_state_dict(best_model_dict)

<All keys matched successfully>

In [42]:
batch_size = 8
sample_size = 8
# test model with no gradient updates
with torch.no_grad():
    correct = 0
    test_loss = []
    test_corr = 0
    labels = []
    pred = []
    counter = 0
    # perform test set evaluation batch wise
    for (X, y) in test_loader:
        # set label to use CUDA if available
        X, y = X.to(device), y.to(device)

        # append original labels
        labels.append(y.view(-1, 1))

        y_pred = best_model(X.view(-1, 3, 225, 225))
        loss = lossfunc(y_pred, y.view(batch_size*sample_size, -1)) # 16 samples in batch, 3 images in one sample

        # get argmax of predicted values, which is our label
        predicted = (y_pred.view(batch_size*sample_size) > .5).long()
        # append predicted label
        pred.append(y_pred)

        # increment correct with correcly predicted labels per batch
        test_corr += (predicted == y.view(batch_size*sample_size)).sum()
        print(test_corr)

        test_loss.append(loss)
        counter += 1
print(f"Test Loss: {test_loss[-1].item():.4f}")
print(f"Test accuracy: {test_corr * 100 / (sample_size*batch_size*counter)}")

tensor(53, device='cuda:0')
tensor(113, device='cuda:0')
tensor(146, device='cuda:0')
tensor(210, device='cuda:0')
tensor(259, device='cuda:0')
tensor(315, device='cuda:0')
tensor(372, device='cuda:0')
tensor(436, device='cuda:0')
Test Loss: 0.0125
Test accuracy: 85.15625
